In [1]:
import pandas as pd

In [2]:
!pip install pymorphy2

In [3]:
!pip install fuzzywuzzy

In [4]:
import pymorphy2
from fuzzywuzzy import fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
df = pd.read_csv('медицинские_протоколы.csv', sep=';')

In [6]:
df.columns

Index(['MedicalRecordKey', 'MedicalRecordDate', 'PatientKey', 'Возраст', 'Пол',
       'СпециальностьВрача', 'Жалобы', 'ПеренесенныеЗаболевания',
       'ПеренесенныеОперации', 'ПринимаемыеПрепараты'],
      dtype='object')

In [7]:
df.isna().sum()

MedicalRecordKey              0
MedicalRecordDate             0
PatientKey                    0
Возраст                       0
Пол                           1
СпециальностьВрача            0
Жалобы                        0
ПеренесенныеЗаболевания     348
ПеренесенныеОперации       3180
ПринимаемыеПрепараты       5794
dtype: int64

In [8]:
df['Пол'] = df['Пол'].fillna(df['Пол'].value_counts().keys()[0])

In [17]:
import pickle

In [18]:
with open('jalobs.pkl', 'rb') as f:
    jalobs = pickle.load(f)

In [20]:
jal = []
for i in jalobs[::-1][:15]:
  jal.append(list(i[1])[0])

In [21]:
jal

['сонливость',
 'нет.',
 'на момент осмотра активных жалоб не предъявляет',
 'заложенность в нем.',
 'на   момент  осмотра   нет.',
 'першение и боль  в горле.',
 'утомляемость .',
 'жалоб нет.',
 'на умеренную хр.боль в правом плечевом суставе',
 'на момент осмотра не предьявляет',
 'жалобы на ощущение тревоги',
 'дискомфорт в верхних отделах живота',
 'периодическое подкашливание',
 'усиливается при ротации',
 'ощущение комка в горле']

In [22]:
jalobs[-2][0] + jalobs[-3][0] + jalobs[-5][0] + jalobs[-8][0] + jalobs[-10][0] > jalobs[-1][0]

True

In [23]:
jalobs[-2][0] + jalobs[-3][0] + jalobs[-5][0] + jalobs[-8][0] + jalobs[-10][0], jalobs[-1][0]

(1094, 442)

In [24]:
jal.insert(0, 'нет')

In [25]:
del jal[2]

In [26]:
del jal[2]

In [27]:
del jal[3]

In [28]:
del jal[5]

In [29]:
del jal[6]

In [30]:
del jal[-1]

In [31]:
len(jal)

10

In [32]:
jal

['нет',
 'сонливость',
 'заложенность в нем.',
 'першение и боль  в горле.',
 'утомляемость .',
 'на умеренную хр.боль в правом плечевом суставе',
 'жалобы на ощущение тревоги',
 'дискомфорт в верхних отделах живота',
 'периодическое подкашливание',
 'усиливается при ротации']

Это ответ на топ 10 самых встречаемых жалоб

--------------------------------------------------------------------

In [44]:
for i in jal:
  df[i] = [0] * df.shape[0]

In [42]:
df['другое'] = [0] * df.shape[0]

In [ ]:
df

In [35]:
#функция собирающая из строки список слов приведенных в нормальную форму

morph = pymorphy2.MorphAnalyzer()

def norma(st):
  s = st.split()
  s1 = []
  for i in s:
    sl = ''
    for j in i:
      if j.isalpha():
        sl += j
    s1.append(sl)
  s2 = []
  for i in s1:
    s2.append(morph.parse(i)[0].normal_form)
  return s2


In [45]:
for i in range(len(df['Жалобы'])):
    norm = norma(df['Жалобы'][i])
    flag = True
    for j in jal:
        for k in norm:
            if fuzz.ratio(k, j) >= 50:
                df[j][i] = 1
                flag = False
    if flag:
      df['другое'][i] = 1

<ipython-input-45-61e5351877df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['другое'][i] = 1
<ipython-input-45-61e5351877df>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[j][i] = 1


In [ ]:
for i in range(len(df['Жалобы'])):
    flag = True
    for j in jal:
        if fuzz.ratio(df['Жалобы'][i], j) >= 60:
            flag = False
            df[j][i] = 1
    if flag:
      df['другое'][i] = 1

In [ ]:
df

In [49]:
for i in jal + ['другое']:
  print(df[i].sum(), i)

3894 нет
6290 сонливость
1187 заложенность в нем.
26 першение и боль  в горле.
1734 утомляемость .
67 на умеренную хр.боль в правом плечевом суставе
31 жалобы на ощущение тревоги
29 дискомфорт в верхних отделах живота
1408 периодическое подкашливание
500 усиливается при ротации
12178 другое


In [50]:
df.to_csv('new_data.csv', index=False)